In [1]:
from keras import Sequential
from keras.layers import Dense,Activation,Conv2D,MaxPool2D,MaxPooling2D,Flatten,Dropout,Input
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
import keras

In [2]:
#vgg-16模型
model = Sequential()

In [3]:
#block 1 224-112
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding='same',name = 'block1_conv1',input_shape=(224,224,3)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same',name = 'block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name ='block1_maxpoling'))

所以这里取出来的filter都是什么样的

In [4]:
#block 2 112-56
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same',name = 'block2_conv1'))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding='same',name = 'block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block2_maxPooling'))

In [5]:
#block 3 56-28
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',name = 'block3_conv1'))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',name = 'block3_conv2'))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',name = 'block3_conv3'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block3_maxPooling'))

In [6]:
#block 4 28-14
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',name = 'block4_conv1'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',name = 'block4_conv2'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',name = 'block4_conv3'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block4_maxPooling'))

In [7]:
#block 5 14-7
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',name = 'block5_conv1'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',name = 'block5_conv2'))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu',padding='same',name = 'block5_conv3'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block5_maxPooling'))

In [8]:
model.add(Flatten())
model.add(Dense(4096,activation='relu',name='fc1'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu',name='fc2'))
model.add(Dropout(0.5))
model.add(Dense(1000,activation='softmax',name='prediction'))

weight 对应的行应该是
(3X3X3+1)X64=1792,直观上感觉3*3*3*64比较能理解，但是不知道，为啥多了以项目
第二个卷积层因为用的是上次的输入，所以输入就是64了，因此得有3*3*64*64，但是summary应该是(3*3*64+1)*64
池化后输入为112*112*64
block2_Conv1:3*3*64*128
block2_Conv2:3*3*128*128

block3_conv1:3*3*128*256
block3_conv2:3*3*256*256
block3_conv3:3*3*256*256

block4_conv1:3*3*256*512
block4_conv2:3*3*512*512
block4_conv3:3*3*512*512

block5_conv1:3*3*512*512
block5_conv2:3*3*512*512
block5_conv3:3*3*512*512

flaten （7*7*512个神经元）
Dense fc1 7*7*512*4096
fc2 4096*4096 但是这里是4097*4096

prediction 4096*1000 这里是4097*1000，感觉是加了一个bias

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_maxpoling (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_maxPooling (MaxPoolin (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [4]:
#预载入的VGG模型
model_pre = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [5]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import time
 
t0 = time.time()
 
img = image.load_img('cat.jpg', target_size = (224, 224))
x = image.img_to_array(img) # 三维（224，224，3）
x = np.expand_dims(x, axis = 0) # 四维（1，224，224，3）
x = preprocess_input(x) # 预处理
print(x.shape)
y_pred = model_pre.predict(x)# 预测概率
 
t1 = time.time()
 
print("测试图：", decode_predictions(y_pred)) # 输出五个最高概率(类名, 语义概念, 预测概率)
print("耗时：", str((t1-t0)*1000), "ms")

(1, 224, 224, 3)
40960/35363 [==================================] - 0s 6us/step
测试图： [[('n02123394', 'Persian_cat', 0.8312202), ('n02123045', 'tabby', 0.024099337), ('n04447861', 'toilet_seat', 0.018717533), ('n04553703', 'washbasin', 0.011249743), ('n03223299', 'doormat', 0.010207416)]]
耗时： 460.4971408843994 ms


In [ ]:
(1, 224, 224, 3)
测试图： [[('n02123045', 'tabby', 0.73477006), ('n02124075', 'Egyptian_cat', 0.07941937), ('n02123159', 'tiger_cat', 0.07054488), ('n02883205', 'bow_tie', 0.019230891), ('n04553703', 'washbasin', 0.013854385)]]
耗时： 538.732959 ms